## AULA 03 - EXTRAÇÃO DE DADOS I - Manipulando Arquivos


## Base de Dados para Nossa AULA

- **API Utilizada:** DummyJson API
- **URL Base**: https://dummyjson.com/auth/
- **Modo de Acesso:** Autenticado


In [ ]:
%pip install requests

In [ ]:
import requests
import json

def authenticate(login, senha):
    login_response = requests.post(
        url="https://dummyjson.com/auth/login",
        headers={"Content-Type": "application/json"},
        data=json.dumps({
            "username": login,
            "password": senha,
            "expiresInMins": 1
        })
    )
    
    return login_response.json()['token']

def get_authorization_headers():
    access_token = authenticate(login="kminchelle", senha="0lelplR")
    
    authorization_header = {
        "Authorization": f"Bearer {access_token}", 
        "Content-Type": "application/json"
    }
    
    return authorization_header

def get_products():
    response = requests.get(
        url="https://dummyjson.com/auth/products", 
        headers=get_authorization_headers()
    )
    
    return response.json()

## Principais Tipos de Arquivos Utilizados em Ambientes de Dados


**CSV** - Comma Separated Values ou Valores Separados por Vírgula

- Arquivos delimitados por um separador, sendo os mais comuns:
  - `,` - Vírgula
  - `;` - Ponto e Vírgula
  - `|` - Pipe
  - `\t` - Tabulação

Os aquivos CSVs possuem básicamente um cabeçalho e seus valores, sendo o cabeçalho delimitado pela primeira linha do arquivo

Suas quebras de linha são definidas por:

- `\n` - Em sistemas Linux/Unix
- `\r\n` - Em sistemas Windows

Algumas Aplicações

- Planilhas Operacioais
- Análise de Dados
- Armazenamento de Dados tabulados de baixa complexidade


## Gravando dados do Tipo CSV com Python e Pandas


## Abordagem Pandas CSV


In [ ]:
%pip install pandas

In [ ]:
### LIVE CODING HERE
import pandas as pd
from io import StringIO

products_response = get_products()["products"]
products_response_json_string = json.dumps(products_response)

df_products = pd.read_json(StringIO(products_response_json_string))
df_products_final = df_products[["id", "title", "description", "price", "stock"]]

df_products_final.to_csv("../clasrroom_03/raw/products_pandas.csv", sep="|", encoding="utf-8", mode="w", index=False)
display(df_products_final)



## Abordagem Python biblioteca CSV


In [ ]:
### LIVE CODING HERE
import csv

products_response = get_products()["products"]

with open("../clasrroom_03/raw/products_pandas_native.csv", mode="w", newline="") as file_to_write:
  fill_names = products_response[0].keys()
  file_write = csv.DictWriter(file_to_write, delimiter=";", quotechar="|", quoting=csv.QUOTE_MINIMAL, fieldnames=fill_names)
  
  file_write.writeheader()
  
  for product in products_response:
    file_write.writerow(product)
    
  file_to_write.flush()
  file_to_write.close()

**JSON** - JavaScript Object Notation

- Arquivos que possuem uma estrutura de Chave valor, podendo estes valores serem:
  - Numeros
  - Texto
  - Booleanos
  - Datas (Representadas como Texto)
  - Objetos
  - Listas

Seus membros são separados por `,` (Vírgula)

Algumas Aplicações

- Trafego de Dados entre Sistemas via API
- Sistemas de Mensageria (KAFKA, Managed Queues, ETC)


## Gravando dados do Tipo JSON com Python e Pandas


## Abordagem Pandas JSON


In [ ]:
### LIVE CODING HERE
import pandas as pd

from io import StringIO

products_response = get_products()["products"]
products_response_json_string = json.dumps(products_response)

df_products = pd.read_json(StringIO(products_response_json_string))
df_products_final = df_products[["id", "title", "description", "price", "stock"]]

df_products_final.to_json("../clasrroom_03/raw/products_pandas.json", orient="records")

## Abordagem Python JSON


In [ ]:
### LIVE CODING HERE
products_response = get_products()["products"]
products_response_filtred = list(map(lambda product: {"id": product["id"], "title": product["title"]}, products_response))

products_response_json_string = json.dumps(products_response_filtred)

with open("../clasrroom_03/raw/products_pandas_native_python.json", mode="w") as file_to_write:
  file_to_write.write(products_response_json_string)  
  file_to_write.flush()
  file_to_write.close()


**PARQUET** - Apache Parquet

- Formato de Arquivo de Código aberto mantido pela Apache

A organização de um arquivo parquet é muito semelhante a organização de uma tabela de um SGBD (MySQL, Oracle, SQL Server, etc) com Linhas e Colunas

- É um formato de Arquivo Colunar
- Possui ótima taxa de compactação

Permite o Armazenamento de Dados Estruturados e Não Estruturados

- ARRAYS
- Estruturas Complexas

Metadados

- Cadas Arquivo Parquet Possui seu Próprio Schema Descrito, sendo um formato de Arquivo Auto-descritivo
- Os metadados do arquivo encontram-se no rodapé do arquivo, segundo a documentação oficial no seguinte formato:

```
    4-byte magic number "PAR1"
    <Column 1 Chunk 1 + Column Metadata>
    <Column 2 Chunk 1 + Column Metadata>
    ...
    <Column N Chunk 1 + Column Metadata>
    <Column 1 Chunk 2 + Column Metadata>
    <Column 2 Chunk 2 + Column Metadata>
    ...
    <Column N Chunk 2 + Column Metadata>
    ...
    <Column 1 Chunk M + Column Metadata>
    <Column 2 Chunk M + Column Metadata>
    ...
    <Column N Chunk M + Column Metadata>
    File Metadata
    4-byte length in bytes of file metadata
    4-byte magic number "PAR1"
```

Utilização

- Sistemas de Armazenamento com Grande Volume de Dados
- Ecossistema BigData

Algumas Vantagens

- Leituras Objetivas e Otimizadas
- Leia apenas o necessário
- Informação sobre os tipos de dados com os quais estamos trabalhando


## Gravando dados do Tipo PARQUET com Pandas


In [ ]:
### LIVE CODING HERE
import pandas as pd

from io import StringIO

products_response = get_products()["products"]
products_response_json_string = json.dumps(products_response)

df_products = pd.read_json(StringIO(products_response_json_string))
df_products_final = df_products[["id", "title", "description", "price", "stock"]]

df_products_final.to_parquet("../clasrroom_03/raw/products_pandas.parquet.gz", compression="gzip", index=False)

# display(pd.read_parquet("../clasrroom_03/raw/products_pandas.parquet.gz"))

### Alguns outros Formatos

- ORC
- AVRO
